In [1]:
#Dependancies
#(this block assumes you've set your mysql password in a config.py folder as a variable named mypass)

import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

from config import mypass

In [2]:
#Establishes a connection with the sqlite database

engine = create_engine('sqlite:///Resources/SMM.db')
Base.metadata.create_all(engine)

In [3]:
#Converts sqlite data into Pandas dataframes

levelsDB = pd.read_sql_query('select * from levels', con=engine)
authorsDB = pd.read_sql_query('select * from authors', con=engine)

In [4]:
#Renames columns for merging

levelsDB_rn = levelsDB.rename(columns={"id": "level_id",
                                       "name": "level_name"})

authorsDB_rn = authorsDB.rename(columns={"id": "author_id",
                                       "name": "author_name"})

In [5]:
#Creates new DBs for levels and authors that only uses the columns we need from each

levelsDB_trn = levelsDB_rn[["theme", "difficulty", "tag", "liked", "played", "shared", "clear_rate",
                           "tries_taken", "tries_success", "author_id"]].copy()

authorsDB_trn = authorsDB_rn[["author_id", "medals", "country"]].copy()

In [6]:
# Set index for each table to be identical

levelsDB_trn.set_index("author_id", inplace=True)

authorsDB_trn.set_index("author_id", inplace=True)

In [7]:
#Creates a merged table of all the mario maker data we need

merge_mario = pd.merge(authorsDB_trn, levelsDB_trn, on="author_id")
clean_mario = merge_mario.dropna()
clean_mario_trn = clean_mario[["medals", "country", "liked", "played"]].copy()

In [8]:
#Creates a table with values for number of levels per country for later use

level_counts = clean_mario_trn["country"].value_counts().tolist()
level_countries = clean_mario_trn["country"].value_counts().index.tolist()

level_list = pd.DataFrame({'country' : level_countries,
 'total_levels' : level_counts
  })
level_list.set_index("country", inplace=True)

In [9]:
#Aggregates and renames data for better readability

mario_countries = clean_mario_trn.groupby(clean_mario_trn['country']).aggregate({'medals': 'sum', 'liked': 'sum', 'played': 'sum'})

mario_countries_rn = mario_countries.rename(columns={"medals": "total_medals",
                                       "liked": "total_likes",
                                                    "played": "total_plays"})

In [10]:
#Merges in the values from the 'number of levels' table created earlier

merge_mario_countries = pd.merge(mario_countries_rn, level_list, on="country")

In [11]:
#Brings in the json file as a pandas dataframe

country_codes = pd.read_json('Resources/country_codes.json')

In [12]:
#Renames columns in both dataframes to make them compatible

country_codes = country_codes.rename(columns={"Name": "country",
                                                "Code": "country_abv"})
country_codes.set_index("country_abv", inplace=True)

merge_mario_countries.index.name = "country_abv"

In [13]:
#Merges our two tables to create a new table with translated abbreviations

mario_codes = pd.merge(country_codes, merge_mario_countries, on="country_abv")

In [14]:
#Reads in the population csv as a pandas dataframe

population_df = pd.read_csv('Resources/popNIndex.csv')

In [15]:
#Removes the columns we don't need from our population dataframe

population_df = population_df[["Country", "Population", "Consumer_price_index"]]

In [16]:
#Renames columns for merge compatability and to fit with lowercase styling

population_df = population_df.rename(columns={"Country": "country",
                                              "Population": "population",
                                              "Consumer_price_index": "consumer_price_index"})

In [17]:
#Merges in the population data with the mario data, but is now missing the country abbreviations

semifinal_mario = pd.merge(population_df, mario_codes, on="country")

In [18]:
#Re-reads and renames the country_codes csv, resetting the earlier changes

country_codes = pd.read_json('Resources/country_codes.json')

country_codes = country_codes.rename(columns={"Name": "country",
                                                "Code": "country_abv"})

In [19]:
#Merges in the country abbreviations from country_codes

final_mario = pd.merge(semifinal_mario, country_codes, on="country")

In [20]:
#Re-organizes the order of the columns for better readability

final_mario = final_mario[["country", "country_abv", "population", "consumer_price_index",
                         "total_medals", "total_likes", "total_plays", "total_levels"]]

In [21]:
#Drops duplicate country entries, keeping the ones with higher population as those are likely more recent

final_mario = final_mario.sort_values(by=["population"], ascending=False)
final_mario = final_mario.drop_duplicates(subset='country',keep='last')

In [22]:
#Resets the final dataframe to be sorted by index again

final_mario = final_mario.sort_index()

In [25]:
final_mario.to_csv("marioData.csv")

In [ ]:
#Establishes a connection to the mysql database
#(this block assumes you've created a mario_db database in mysql)

connection_string = f"root:{mypass}@localhost/mario_db"
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
#Pushes the finished mario dataframe to the mysql database as a table

final_mario.to_sql(name='mario_table',\
                           con=engine, if_exists='append', index=False)